In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath("../../data"))

from text_data import OpusBooksDataset

In [2]:
dataset = OpusBooksDataset()
data = dataset.process_dataset()

Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


Combining:   0%|          | 0/94 [00:00<?, ?ba/s]

Tokenizing:   0%|          | 0/94 [00:00<?, ?ba/s]

Filtering:   0%|          | 0/94 [00:00<?, ?ba/s]

Filtering:   0%|          | 0/94 [00:00<?, ?ba/s]

Splitting:   0%|          | 0/79 [00:00<?, ?ba/s]

In [16]:
from torch.utils.data import DataLoader, BatchSampler, default_collate

class PaddingSampler(BatchSampler):
    def __init__(self, lengths):
        indices = torch.arange(len(lengths))
        self.length_idx = torch.vstack((indices, lengths)).T

    def __iter__(self):
        # Shuffle the indices
        self.length_idx = self.length_idx[torch.randperm(len(self.length_idx))]
        # Resort, this will get a random order within each length group
        self.length_idx = self.length_idx[self.length_idx[:, 1].argsort()]
        for i in self.length_idx[:, 0]:
            yield i.item()

    def __len__(self) -> int:
        return len(self.length_idx)

def pad_collate(batch):
    # Get max lengths in the batch
    max_es_len = max([len(x["es_ids"]) for x in batch])
    max_en_len = max([len(x["en_ids"]) for x in batch])
    # Pad batch to max length
    for i in range(len(batch)):
        batch[i]["es_ids"] = torch.cat(
            [batch[i]["es_ids"], torch.zeros(max_es_len - len(batch[i]["es_ids"]))]
        )
        batch[i]["en_ids"] = torch.cat(
            [batch[i]["en_ids"], torch.zeros(max_en_len - len(batch[i]["en_ids"]))]
        )
    return default_collate(batch)

In [17]:
train = DataLoader(data, batch_size=1024, sampler=PaddingSampler(data["en_lens"]), collate_fn=pad_collate)
d = iter(train)

In [24]:
n = next(d)

In [27]:
n["en_ids"][0]

tensor([2793.,  165., 1479.,  175.,  145.,  245.,  252., 3668.,  194.,  143.,
        2473., 4863.])

In [32]:
torch.roll(n["en_ids"], 1, -1)[0]

tensor([4863., 2793.,  165., 1479.,  175.,  145.,  245.,  252., 3668.,  194.,
         143., 2473.])

In [118]:
len(train)

49

In [23]:
indices = torch.arange(len(data))
lengths = data["en_lens"]
length_idx = torch.vstack((indices, lengths)).T


In [130]:
from torch.utils.data import SequentialSampler

s = PaddingSampler(data["en_lens"][:1000])
for i in s:
    print(i)

Shuffling
644
613
715
828
191
718
945
478
3
627
619
626
178
629
269
761
987
830
994
272
13
844
464
253
854
456
840
892
959
165
657
817
685
724
67
816
572
216
785
853
83
14
174
802
919
700
536
977
517
727
906
944
556
488
477
37
979
834
786
751
486
615
91
679
81
664
388
396
411
565
911
633
328
674
955
965
762
449
503
513
484
143
891
295
466
369
440
735
120
887
742
864
285
827
51
963
648
803
306
507
746
108
600
699
60
590
85
277
686
655
719
497
707
593
93
491
171
843
877
460
474
650
133
659
585
470
649
372
292
522
170
153
418
925
790
427
307
248
438
58
333
407
525
485
728
697
75
220
452
391
747
885
445
562
278
693
415
353
124
638
487
444
745
939
127
274
275
117
902
714
592
155
291
656
157
623
672
190
607
957
570
773
684
591
681
0
576
208
815
261
579
420
308
918
417
123
227
789
673
273
645
492
748
784
515
958
309
126
385
850
914
862
315
239
855
883
301
110
162
281
552
441
476
498
580
39
766
386
872
52
956
380
90
760
173
779
881
770
335
920
43
314
978
298
132
731
631
300
457
639
142
455
414